**Problem 1045:** Customers Who Bought All Products  
Link: https://leetcode.com/problems/customers-who-bought-all-products/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
-- Using subquery
SELECT base.customer_id
FROM 
(
    SELECT customer_id, COUNT(DISTINCT product_key) AS products_bought
    FROM Customer
    GROUP BY customer_id
) base
WHERE base.products_bought = (SELECT COUNT(product_key) FROM Product);

-- Using CTE
WITH base AS
(
    SELECT customer_id, COUNT(DISTINCT product_key) AS products_bought
    FROM Customer
    GROUP BY customer_id
),
total_products AS
(
    SELECT COUNT(product_key) AS total_products FROM Product
)
SELECT base.customer_id
FROM base, total_products
WHERE base.products_bought = total_products.total_products;


**Problem 619:** Biggest Single Number  
Link: https://leetcode.com/problems/biggest-single-number/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
-- Intuition is to apply the condition to have count = 1, then order by and limit
-- Simple query
SELECT IFNULL(
    (
        SELECT num
        FROM MyNumbers
        GROUP BY num
        HAVING COUNT(num) = 1
        ORDER BY num DESC
        LIMIT 1
    ), NULL
) AS num

-- Now using CTE
WITH NumberCounts AS (
    SELECT num
    FROM MyNumbers
    GROUP BY num
    HAVING COUNT(num) = 1
)
SELECT IFNULL(
    (
        SELECT num
        FROM NumberCounts
        ORDER BY num DESC
        LIMIT 1
    ), NULL
) AS num

**Problem 550:** Game Play Analysis IV  
Link: https://leetcode.com/problems/game-play-analysis-iv/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
-- Intuition is that we first find the minimum date for each player and somehow 
-- check that the next date, for the given player is present in the database
SELECT ROUND(COUNT(base.player_id) / (SELECT COUNT(DISTINCT a1.player_id) FROM Activity AS a1),2) as fraction
FROM 
(
    SELECT player_id, MIN(event_date) as first_logged
    FROM Activity
    GROUP BY player_id
) base
-- check next date is present for the player
WHERE EXISTS (
    SELECT 1
    FROM Activity
    WHERE player_id = base.player_id
    AND event_date = DATE_ADD(base.first_logged, INTERVAL 1 DAY)
)

-- Approach using JOIN (this is also not optimal)
-- The broad step by step approach of finding the minimum date AND THEN the next day will still be the same
SELECT 
    ROUND(
        (SELECT COUNT(DISTINCT a2.player_id)
         FROM Activity a2
         JOIN Activity a3 ON a2.player_id = a3.player_id
         WHERE DATE_ADD(a2.event_date, INTERVAL 1 DAY) = a3.event_date
         AND a2.event_date = ( -- Find the minimum date for the player - this is exactly one value 
             SELECT MIN(event_date) 
             FROM Activity 
             WHERE player_id = a2.player_id
         )
        ) / 
        (SELECT COUNT(DISTINCT a1.player_id) FROM Activity AS a1),
        2
    ) AS fraction;


-- Using CTEs 
-- Intuition is we first find the FirstLogin and then find if NextDayLogin exists for the player
WITH FirstLogin AS (
    SELECT player_id, MIN(event_date) as first_logged
    FROM Activity
    GROUP BY player_id
),
NextDayLogin AS (
    SELECT fl.player_id 
    FROM FirstLogin fl
    JOIN Activity a ON fl.player_id = a.player_id
    WHERE a.event_date = DATE_ADD(fl.first_logged, INTERVAL 1 DAY)
)
SELECT ROUND(COUNT(DISTINCT ndl.player_id)/COUNT(DISTINCT fl.player_id),2) as fraction
FROM FirstLogin fl
LEFT JOIN NextDayLogin ndl ON fl.player_id = ndl.player_id;

**Problem 1174:** Immediate Food Delivery II  
Link: https://leetcode.com/problems/immediate-food-delivery-ii/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
-- My solution: using window function

WITH first_orders AS (
    SELECT
        customer_id,
        order_date AS first_order_date,
        customer_pref_delivery_date
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) AS rn 
        -- a partition is created based on customer id, order dates are ordered in ascending order and then row number is assigned to each row
        -- rn = 1 means the first order of each customer is selected 
        -- this is a useful paradigm because it allows us to get the nth order of each customer
    FROM
        Delivery
)
SELECT
    ROUND(AVG(CASE WHEN first_order_date = customer_pref_delivery_date THEN 1 ELSE 0 END) * 100, 2) AS immediate_percentage
FROM
    first_orders
WHERE
    rn = 1;

-- My solution: using sub-query
SELECT 
    ROUND(SUM(d2.first_order_date = d2.pref_date)/COUNT(d2.first_order_date) * 100, 2) AS immediate_percentage
    FROM
    (
        SELECT 
            customer_id, 
            MIN(d1.order_date) AS first_order_date,
            MIN(d1.customer_pref_delivery_date) AS pref_date
        FROM 
            Delivery AS d1
        GROUP BY 
            customer_id
    ) AS d2;

-- Most voted solution
SELECT 
    ROUND(AVG(order_date = customer_pref_delivery_date) * 100, 2) AS immediate_percentage
FROM 
    Delivery
WHERE 
    (customer_id, order_date) IN (
        SELECT 
            customer_id, 
            MIN(order_date) 
        FROM 
            Delivery 
        GROUP BY 
            customer_id
    );

**Problem 1193:** Monthly Transactions I  
Link: https://leetcode.com/problems/monthly-transactions-i/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
-- Using CASE Statement to navigate this problem
-- Works without CASE too because approved is binary so SUM((state='approved')*amount) will work too; however this is more error prone
SELECT DATE_FORMAT(trans_date, '%Y-%m') AS month,
country,
COUNT(*) AS trans_count,
SUM(amount) AS trans_total_amount,
SUM(CASE WHEN state = 'approved' THEN 1 ELSE 0 END) AS approved_count,
SUM(CASE WHEN state = 'approved' THEN amount ELSE 0 END) AS approved_total_amount,
FROM 
Transactions
GROUP BY month, country

**Problem 1934:** Confirmation Rate  
Link: https://leetcode.com/problems/confirmation-rate/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
-- First method using simple LEFT JOIN
SELECT s.user_id,
ROUND(
    IFNULL(SUM(c.action='confirmed')/count(c.user_id),0),2) as confirmation_rate
FROM signups AS s
LEFT JOIN confirmations AS c
ON s.user_id = c.user_id
GROUP BY c.user_id;

-- Second method using window function
-- A clear reason for why this is better is because this can handle addition of metrics in a much better way (in the CTE)
WITH ConfirmationCounts AS (
    SELECT user_id,
    SUM(CASE WHEN action = 'confirmed' THEN 1 ELSE 0 END) OVER (PARTITION BY user_id) AS confirmed_count,
    COUNT(action) OVER (PARTITION BY user_id) AS total_count
    FROM confirmations
)
SELECT s.user_id,
ROUND(
    IFNULL(confirmed_count/total_count,0),2) as confirmation_rate
FROM signups AS s
LEFT JOIN ConfirmationCounts cc ON s.user_id = cc.user_id
GROUP BY s.user_id, cc.confirmed_count, cc.total_count;

**Problem 570:** Managers with at least 5 Direct Reports  
Link: https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/?envType=study-plan-v2&envId=top-sql-50

In [ ]:
-- Method 1 uses JOIN
SELECT e1.name FROM Employee AS e1
JOIN Employee AS e2
ON e1.id = e2.managerId
GROUP BY e2.managerId
HAVING COUNT(e1.id) > 4;

-- Method 2 uses subquery
SELECT name FROM Employee e1
WHERE e1.id IN (
    SELECT managerId FROM Employee
    GROUP BY managerId
    HAVING COUNT(id) > 4
);

**Problem 1280:** Students and Examinations  
Link: https://leetcode.com/problems/students-and-examinations/description/?envType=study-plan-v2&envId=top-sql-50  
Useful concept(s):  
- Cross join - to get a view where we want everything

In [ ]:
# Query
SELECT st.student_id, st.student_name, su.subject_name, COALESCE(e.attended_exams,0) as attended_exams
FROM Students as st
CROSS JOIN Subjects as su
LEFT JOIN 
(SELECT student_id, subject_name, COUNT(*) as attended_exams
FROM Examinations
GROUP BY student_id, subject_name) as e
ON st.student_id = e.student_id AND su.subject_name = e.subject_name
ORDER BY st.student_id, su.subject_name;